In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in cuisine_ingredients csv file
column_names = ["cuisine", "recipe", "ingredients", "full_ingredients", "image_url"]
df = pd.read_csv("cuisine_ingredients.csv", names=column_names, encoding='utf-16')
df.head()

,cuisine,recipe,ingredients,full_ingredients,image_url
0,African,Moroccan-style roasted veg,"['peppers', 'sweet potatoes', 'red onions', 'c...","['2 large red or orange peppers, de-seeded and...",https://ichef.bbci.co.uk/food/ic/food_16x9_832...
1,African,"Beef, black-eyed bean and plantain hotpot","['sirloin', 'vegetable oil', 'onion', 'garlic'...","['350g/12oz dried black-eyed beans', '900g/2lb...",https://ichef.bbci.co.uk/food/ic/food_16x9_832...
2,African,Bobotie,"['olive oil', 'onion', 'garlic', 'lamb mince',...","['50ml/2fl oz olive oil', '1 onion, chopped', ...",NaN
3,African,South African bobotie,"['butter', 'milk', 'breadcrumbs', 'eggs', 'oli...","['butter, for greasing', '400ml/14fl oz full-f...",NaN
4,African,Bunny chow,"['olive oil', 'onion', 'garlic', 'chilli', 'gi...","['1 tbsp olive oil', '1 onion, thinly sliced',...",NaN


In [3]:
df.drop_duplicates()

,cuisine,recipe,ingredients,full_ingredients,image_url
0,African,Moroccan-style roasted veg,"['peppers', 'sweet potatoes', 'red onions', 'c...","['2 large red or orange peppers, de-seeded and...",https://ichef.bbci.co.uk/food/ic/food_16x9_832...
1,African,"Beef, black-eyed bean and plantain hotpot","['sirloin', 'vegetable oil', 'onion', 'garlic'...","['350g/12oz dried black-eyed beans', '900g/2lb...",https://ichef.bbci.co.uk/food/ic/food_16x9_832...
2,African,Bobotie,"['olive oil', 'onion', 'garlic', 'lamb mince',...","['50ml/2fl oz olive oil', '1 onion, chopped', ...",NaN
3,African,South African bobotie,"['butter', 'milk', 'breadcrumbs', 'eggs', 'oli...","['butter, for greasing', '400ml/14fl oz full-f...",NaN
4,African,Bunny chow,"['olive oil', 'onion', 'garlic', 'chilli', 'gi...","['1 tbsp olive oil', '1 onion, thinly sliced',...",NaN
...,...,...,...,...,...
4719,Turkish and Middle Eastern,"Warm, spiced oranges with labneh","['yoghurt', 'oranges', 'cinnamon', 'star anise...",['2 x 150g/5½oz pots fat-free natural yoghurt'...,https://ichef.bbci.co.uk/food/ic/food_16x9_832...
4720,Turkish and Middle Eastern,Spiced chicken flatbread with Lebanese chicke...,"['vegetable oil', 'plain flour', 'semolina', '...","['a drizzle vegetable oil, for greasing', '400...",NaN
4721,Turkish and Middle Eastern,Fesenjoon,"['walnut', 'molasses', 'tomato purée', 'cinnam...","['250g/9oz walnut halves', '2 ice cubes', '125...",https://ichef.bbci.co.uk/food/ic/food_16x9_832...
4722,Turkish and Middle Eastern,Za’atar cod with relish,"['olive oil', 'cod', 'sea salt', 'olives', 'pr...","['2 heaped tbsp Za’atar ', '4 tbsp olive oil, ...",https://ichef.bbci.co.uk/food/ic/food_16x9_832...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4724 entries, 0 to 4723
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   cuisine           4724 non-null   object
 1   recipe            4724 non-null   object
 2   ingredients       4724 non-null   object
 3   full_ingredients  4724 non-null   object
 4   image_url         2515 non-null   object
dtypes: object(5)
memory usage: 184.7+ KB


In [5]:
# Transform cuisine and ingredients into a seperate table
df[["cuisine", "recipe", "ingredients"]].to_csv("cuisine_ingredients_short.csv", index=False)